In [1]:
import pandas as pd
import numpy as np
import sqlite3
import os
from fractions import Fraction

def get_ratio(ins,outs):    
    """ Function to display in to out ratio in the format 1:7 """
    if ins==0:
        ratio='0'+':'+str(int(outs))
        return ratio
    elif outs==0:
        ratio=str(int(ins))+':'+'0'
        return ratio
    elif ins==outs:
        ratio='1:1'
        return ratio
    else:
        ratio=str(Fraction(ins/outs).limit_denominator(1000)).split('/')[0]+':'+str(Fraction(ins/outs).limit_denominator(1000)).split('/')[1]
        return ratio

In [2]:
data_path='/Users/anastasiaclark/irs_nyc_migration/data'
db='irsmig_county_database'
con = sqlite3.connect(os.path.join(data_path,db,"irs_migration_county.sqlite"))
metros=pd.read_csv(os.path.join(data_path,'metros','metros_basic.csv'),converters={'fips':str,'co_code':str,'cbsa_code':str})

In [3]:
years=['2011_12','2012_13','2013_14','2014_15'] # project years 

# for year in years:
year=years[3]
table1='outflow_{}'.format(year)
table2='inflow_{}'.format(year)

In [4]:
df_out = pd.read_sql_query("SELECT * from {} where {}.origin!={}.destination".format(table1, table1, table1), con)
df_in=pd.read_sql_query("SELECT * from {} where {}.origin!={}.destination".format(table2, table2, table2), con)      

In [5]:
# Make the index to be same for same records
df_out['uid']=df_out.origin+"_"+df_out.destination
df_in['uid']=df_in.origin+"_"+df_in.destination
df_in.set_index('uid', inplace=True)
df_out.set_index('uid', inplace=True)

In [6]:
# NYC counties
nyc=['36005','36047','36061','36081','36085']

In [7]:
nyc_in=df_in[(df_in['destination'].isin(nyc)) & (~df_in['origin'].isin(nyc))]
nyc_out=df_out[(df_out['origin'].isin(nyc)) & (~df_out['destination'].isin(nyc))]

In [8]:
to_nyc=nyc_in[['origin','co_orig_name','exemptions','st_orig_abbrv']].groupby(['origin','co_orig_name','st_orig_abbrv']).sum().sort_values('exemptions',ascending=False).reset_index()
from_nyc=nyc_out[['destination','co_dest_name','st_dest_abbrv','exemptions']].groupby(['destination','co_dest_name','st_dest_abbrv']).sum().sort_values('exemptions',ascending=False).reset_index()
to_nyc.rename(columns={'origin':'co_fips','co_orig_name':'co_name','exemptions':'inflow','st_orig_abbrv':'state'},inplace=True)
from_nyc.rename(columns={'destination':'co_fips','co_dest_name':'co_name','st_dest_abbrv':'state','exemptions':'outflow'},inplace=True)

In [9]:
flows_nyc = to_nyc.merge(from_nyc, on=['co_fips', 'co_name', 'state'], how='outer')

In [10]:
flows_nyc[['co_name','state','inflow']].sort_values('inflow', ascending=False).head(n=10)

,co_name,state,inflow
0,Other flows - Different State,DS,19743.0
1,Nassau County,NY,11900.0
2,Westchester County,NY,9048.0
3,Suffolk County,NY,5837.0
4,Hudson County,NJ,4709.0
5,Bergen County,NJ,3860.0
6,Los Angeles County,CA,3465.0
7,Travis County,TX,3067.0
8,Fairfield County,CT,2966.0
9,Foreign - Overseas,FR,2872.0


In [11]:
# most of the top senders to NYC are also top receivers of migrants from NYC;
flows_nyc[['co_name','state','outflow']].sort_values('outflow',ascending=False).head(n=10)

,co_name,state,outflow
0,Other flows - Different State,DS,23807.0
1,Nassau County,NY,20316.0
2,Westchester County,NY,13986.0
3,Suffolk County,NY,8171.0
4,Hudson County,NJ,7273.0
9,Foreign - Overseas,FR,6666.0
5,Bergen County,NJ,5816.0
6,Los Angeles County,CA,5365.0
10,Essex County,NJ,4791.0
8,Fairfield County,CT,4283.0


In [12]:
# merge metro areas info to nyc flows to determine what counties from thesae flows are urban
flows_nyc=flows_nyc.merge(metros[['cbsa_name','fips']], left_on='co_fips', right_on='fips', how='left').drop('fips',1)

In [13]:
flows_nyc.head()

,co_fips,co_name,state,inflow,outflow,cbsa_name
0,59000,Other flows - Different State,DS,19743.0,23807.0,NaN
1,36059,Nassau County,NY,11900.0,20316.0,"New York-Newark-Jersey City, NY-NJ-PA"
2,36119,Westchester County,NY,9048.0,13986.0,"New York-Newark-Jersey City, NY-NJ-PA"
3,36103,Suffolk County,NY,5837.0,8171.0,"New York-Newark-Jersey City, NY-NJ-PA"
4,34017,Hudson County,NJ,4709.0,7273.0,"New York-Newark-Jersey City, NY-NJ-PA"


In [14]:
# calculate fractions, net flow and ratio
flows_nyc['net_flow'] = flows_nyc.inflow - flows_nyc.outflow
flows_nyc['in_ratio'] = flows_nyc.inflow / flows_nyc.outflow
flows_nyc['out_ratio'] = flows_nyc.outflow / flows_nyc.inflow
# need to fill Nulls with 0 for in and out flow to apply get_ration function
#flows_nyc.inflow.fillna(0, inplace=True)
#flows_nyc.outflow.fillna(0, inplace=True)
#flows_nyc['in_to_out_ratio'] = flows_nyc.apply(lambda x: get_ratio(x['inflow'], x['outflow']), axis=1)
flows_nyc.head()

,co_fips,co_name,state,inflow,outflow,cbsa_name,net_flow,in_ratio,out_ratio
0,59000,Other flows - Different State,DS,19743.0,23807.0,NaN,-4064.0,0.829294,1.205845
1,36059,Nassau County,NY,11900.0,20316.0,"New York-Newark-Jersey City, NY-NJ-PA",-8416.0,0.585745,1.707227
2,36119,Westchester County,NY,9048.0,13986.0,"New York-Newark-Jersey City, NY-NJ-PA",-4938.0,0.646933,1.545756
3,36103,Suffolk County,NY,5837.0,8171.0,"New York-Newark-Jersey City, NY-NJ-PA",-2334.0,0.714356,1.399863
4,34017,Hudson County,NJ,4709.0,7273.0,"New York-Newark-Jersey City, NY-NJ-PA",-2564.0,0.647463,1.544489


In [15]:
# drop a column in each table, so that the columns are the same in both tables
df_in.drop('co_orig_name',1,inplace=True)
df_out.drop('co_dest_name',1,inplace=True)

In [16]:
# get inflow and outflow into a single table and since most of the records exist in both tables, drop duplicates
flows=pd.concat([df_in,df_out],axis=0).drop_duplicates(subset=['origin','destination','returns','exemptions'])

In [17]:
# check if there are any records with duplicated indexes
len(flows[flows.index.duplicated()])

94

In [18]:
# 94 records have different data in inflow and outflow tables
# get the indices of the non-matching records
non_matching=flows.index[flows.index.duplicated()]

In [19]:
# look at 10 of them
flows.loc[non_matching].head(n=10)

,destination,disclosure,exemptions,income,origin,returns,st_dest_abbrv,st_orig_abbrv
uid,,,,,,,,
06001_06083,06083,NaN,108.0,3725.0,06001,73.0,CA,CA
06001_06083,06083,NaN,112.0,2469.0,06001,74.0,CA,CA
06013_06085,06085,NaN,1363.0,256538.0,06013,789.0,CA,CA
06013_06085,06085,NaN,1359.0,70854.0,06013,788.0,CA,CA
06013_32031,32031,NaN,266.0,155568.0,06013,142.0,NV,CA
06013_32031,32031,NaN,261.0,22906.0,06013,141.0,NV,CA
06013_06019,06019,NaN,188.0,9401.0,06013,85.0,CA,CA
06013_06019,06019,NaN,186.0,3576.0,06013,84.0,CA,CA
06013_48113,48113,NaN,104.0,12731.0,06013,52.0,TX,CA


In [20]:
# looks like the differences in returns and exemptions is small
# compare the difference in returns and exemptions for each of these records
# create a single dataframe of differences by combining each difference between 2 rows
dfs=[]
for ix in non_matching:
    a=flows.loc[ix,['exemptions','returns']]
    d=a.diff()
    dfs.append(d)
df=pd.concat(dfs)

In [21]:
# difference in exemptions ranges from -6 to 7 and difference in returns ranges from -2 to 2
print 'Max difference in exemptions', df.exemptions.max()
print 'Min difference in exemptions',df.exemptions.min()
print 'Max difference in returns',df.returns.max()
print 'Min difference in returns',df.returns.min()

Max difference in exemptions 7.0
Min difference in exemptions -6.0
Max difference in returns 2.0
Min difference in returns -2.0


In [22]:
# since the difference is not large, drop one of the duplicated indices from the combined dataframe
print len(flows)
flows = flows[~flows.index.duplicated(keep='first')]
print '94 records have been dropped'
print len(flows)

54573
94 records have been dropped
54479


In [23]:
# merge metro area information twice: for county of origin and for county of destination
flows = flows.merge(metros[['fips', 'cbsa_name']],left_on='destination', right_on='fips', how='left').drop('fips',1).rename(columns={'cbsa_name':'dest_cbsa'})
flows = flows.merge(metros[['fips', 'cbsa_name']],left_on='origin', right_on='fips', how='left').drop('fips',1).rename(columns={'cbsa_name':'orig_cbsa'})

In [24]:
# groupby metro of origin and destination to get flows b/n metro areas
metro_flows=flows[['orig_cbsa','dest_cbsa','returns','exemptions']].groupby(['orig_cbsa','dest_cbsa']).sum().reset_index()

In [25]:
# some of the largest flows are happening within metro areas
metro_flows.sort_values(by='exemptions', ascending=False).head(n=20)

,orig_cbsa,dest_cbsa,returns,exemptions
8396,"New York-Newark-Jersey City, NY-NJ-PA","New York-Newark-Jersey City, NY-NJ-PA",233212.0,405566.0
501,"Atlanta-Sandy Springs-Roswell, GA","Atlanta-Sandy Springs-Roswell, GA",93406.0,185267.0
3225,"Dallas-Fort Worth-Arlington, TX","Dallas-Fort Worth-Arlington, TX",80159.0,154677.0
13194,"Washington-Arlington-Alexandria, DC-VA-MD-WV","Washington-Arlington-Alexandria, DC-VA-MD-WV",81815.0,145500.0
2291,"Chicago-Naperville-Elgin, IL-IN-WI","Chicago-Naperville-Elgin, IL-IN-WI",72249.0,137514.0
5279,"Houston-The Woodlands-Sugar Land, TX","Houston-The Woodlands-Sugar Land, TX",52904.0,108839.0
3511,"Denver-Aurora-Lakewood, CO","Denver-Aurora-Lakewood, CO",51804.0,96175.0
9269,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD","Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",47191.0,80801.0
7770,"Minneapolis-St. Paul-Bloomington, MN-WI","Minneapolis-St. Paul-Bloomington, MN-WI",44241.0,76247.0
1475,"Boston-Cambridge-Newton, MA-NH","Boston-Cambridge-Newton, MA-NH",46008.0,71405.0


In [26]:
ny_metro_in=metro_flows[(metro_flows.dest_cbsa.str.contains('New York')) & (~metro_flows.orig_cbsa.str.contains('New York'))].copy()
ny_metro_out=metro_flows[(metro_flows.orig_cbsa.str.contains('New York')) & (~metro_flows.dest_cbsa.str.contains('New York'))].copy()

In [27]:
print ny_metro_in.head()
print ny_metro_out.head()

                             orig_cbsa                              dest_cbsa  \
55                           Akron, OH  New York-Newark-Jersey City, NY-NJ-PA   
122        Albany-Schenectady-Troy, NY  New York-Newark-Jersey City, NY-NJ-PA   
183                    Albuquerque, NM  New York-Newark-Jersey City, NY-NJ-PA   
246  Allentown-Bethlehem-Easton, PA-NJ  New York-Newark-Jersey City, NY-NJ-PA   
374                      Ann Arbor, MI  New York-Newark-Jersey City, NY-NJ-PA   

     returns  exemptions  
55      44.0        67.0  
122   1064.0      1543.0  
183     49.0        67.0  
246   1948.0      3288.0  
374    122.0       146.0  
                                  orig_cbsa  \
8324  New York-Newark-Jersey City, NY-NJ-PA   
8325  New York-Newark-Jersey City, NY-NJ-PA   
8326  New York-Newark-Jersey City, NY-NJ-PA   
8327  New York-Newark-Jersey City, NY-NJ-PA   
8328  New York-Newark-Jersey City, NY-NJ-PA   

                              dest_cbsa  returns  exemptions  
8324 

In [28]:
ny_metro_in=ny_metro_in[['orig_cbsa','exemptions']].rename(columns={'orig_cbsa':'metro_name','exemptions':'inflow'})
ny_metro_out=ny_metro_out[['dest_cbsa','exemptions']].rename(columns={'dest_cbsa':'metro_name','exemptions':'outflow'})

In [29]:
ny_by_metro=pd.merge(ny_metro_in, ny_metro_out, on='metro_name', how='outer')
ny_by_metro.head(n=10)

,metro_name,inflow,outflow
0,"Akron, OH",67.0,NaN
1,"Albany-Schenectady-Troy, NY",1543.0,2626.0
2,"Albuquerque, NM",67.0,NaN
3,"Allentown-Bethlehem-Easton, PA-NJ",3288.0,6069.0
4,"Ann Arbor, MI",146.0,122.0
5,"Asheville, NC",32.0,64.0
6,"Atlanta-Sandy Springs-Roswell, GA",2615.0,6628.0
7,"Atlantic City-Hammonton, NJ",920.0,718.0
8,"Austin-Round Rock, TX",5231.0,6245.0
9,"Baltimore-Columbia-Towson, MD",2996.0,2903.0


In [30]:
# since some metro areas don't send or don't recieve migrants from NY metro, replace NaN with 0
ny_by_metro.fillna(0,inplace=True)
ny_by_metro['net_flow']=ny_by_metro.inflow-ny_by_metro.outflow

In [31]:
# top 10 senders to NY metro have negative net flow 
ny_by_metro.sort_values(['inflow','outflow'],ascending=False).head(n=10)

,metro_name,inflow,outflow,net_flow
81,"Philadelphia-Camden-Wilmington, PA-NJ-DE-MD",9907.0,13870.0,-3963.0
65,"Miami-Fort Lauderdale-West Palm Beach, FL",9757.0,18596.0,-8839.0
122,"Washington-Arlington-Alexandria, DC-VA-MD-WV",7660.0,8391.0,-731.0
60,"Los Angeles-Long Beach-Anaheim, CA",6196.0,9626.0,-3430.0
16,"Bridgeport-Stamford-Norwalk, CT",5901.0,9683.0,-3782.0
8,"Austin-Round Rock, TX",5231.0,6245.0,-1014.0
14,"Boston-Cambridge-Newton, MA-NH",5001.0,6138.0,-1137.0
116,"Trenton, NJ",3962.0,4805.0,-843.0
3,"Allentown-Bethlehem-Easton, PA-NJ",3288.0,6069.0,-2781.0
23,"Chicago-Naperville-Elgin, IL-IN-WI",3283.0,3761.0,-478.0


In [32]:
con.close()